# Diabetic Retinopathy Detection



In [ ]:
from google.colab import drive 


drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Importing the libraries

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.11.0'

In [ ]:
tf.random.set_seed(13)

In [ ]:
!nvidia-smi

Fri Mar 10 00:13:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Part 1 - Data Preprocessing



### Image Augmentation

In [ ]:
import os
import random
import imgaug as ia
from imgaug import augmenters as iaa
import imageio
import time

# define augmentation pipeline
aug_pipeline = iaa.Sequential([
    iaa.Fliplr(p=0.5),
    iaa.Flipud(p=0.5),
    iaa.Rotate((-45, 45)),
    iaa.Crop(percent=(0, 0.2))
])

# define path to the folder containing the images
path_to_folder = '/content/gdrive/MyDrive/saunar/positive'

# loop through all the images in the folder
for filename in os.listdir(path_to_folder):
      # load the image
      img = imageio.imread(os.path.join(path_to_folder, filename))

      # randomly generate the number of augmentations to apply
      num_augmentations = random.randint(1, 4)
      for i in range(num_augmentations):
          augmented_img = aug_pipeline.augment_image(img)

          # save the augmented image with a modified filename
          timestamp = str(int(time.time())) # generate unique identifier
          new_filename = "aug_" + filename[:-4] + "_" + timestamp + ".jpg"
          imageio.imwrite(os.path.join(path_to_folder, new_filename), augmented_img)

### Preprocessing the Training and Testing set

In [ ]:
!rm -r  '/content/gdrive/MyDrive/saunar/.ipynb_checkpoints'

rm: cannot remove '/content/gdrive/MyDrive/saunar/.ipynb_checkpoints': No such file or directory


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=20,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)

training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/saunar',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'binary',
                                                 subset="training")

test_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/saunar',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'binary',
                                                 subset="validation")

Found 4303 images belonging to 2 classes.
Found 1075 images belonging to 2 classes.


In [ ]:
test_set.class_indices

{'negative': 0, 'positive': 1}

In [ ]:
import numpy as np

### Preprocessing the Test set

## Part 2 - Building the model



In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:

from keras.layers import Input
from keras.applications import ResNet50


In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224,224, 3)))

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
cnn.add(base_model)

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the model

In [ ]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [ ]:
cnn.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy',
                                                                      tf.keras.metrics.Precision(),
                                                                      tf.keras.metrics.Recall(),
                                                                      keras.metrics.AUC(),
                                                                      tf.keras.metrics.TruePositives(),
                                                                      tf.keras.metrics.TrueNegatives(),
                                                                      tf.keras.metrics.FalsePositives(),
                                                                      tf.keras.metrics.FalseNegatives(),
                                                                      ])


In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 32)          589856    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                        

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 20)

Epoch 1/20
135/135 [==============================] - 1624s 12s/step - loss: 0.2777 - accuracy: 0.9010 - precision: 0.8985 - recall: 0.7638 - auc: 0.9269 - true_positives: 1009.0000 - true_negatives: 2868.0000 - false_positives: 114.0000 - false_negatives: 312.0000 - val_loss: 1.3885 - val_accuracy: 0.3070 - val_precision: 0.3070 - val_recall: 1.0000 - val_auc: 0.5352 - val_true_positives: 330.0000 - val_true_negatives: 0.0000e+00 - val_false_positives: 745.0000 - val_false_negatives: 0.0000e+00
Epoch 2/20
135/135 [==============================] - 460s 3s/step - loss: 0.1603 - accuracy: 0.9489 - precision: 0.9553 - recall: 0.8743 - auc: 0.9710 - true_positives: 1155.0000 - true_negatives: 2928.0000 - false_positives: 54.0000 - false_negatives: 166.0000 - val_loss: 4.1345 - val_accuracy: 0.3070 - val_precision: 0.3070 - val_recall: 1.0000 - val_auc: 0.5000 - val_true_positives: 330.0000 - val_true_negatives: 0.0000e+00 - val_false_positives: 745.0000 - val_false_negatives: 0.0000e+00
E